In [118]:
import cv2
import numpy as np
from cv2 import VideoWriter, VideoWriter_fourcc

In [119]:
# Create a light gray image
frame = cv2.imread(filename='white_background.jpg')
car = cv2.imread(filename='car.png')
car = cv2.resize(car,(240,300))
bus = cv2.imread(filename='bus.png')
bus = cv2.resize(bus,(220,380))
car2 = cv2.imread(filename='car2.png')
car2 = cv2.resize(car2,(180,300))
x_offset_car = 900
y_offset_car = 700
x_offset_bus = 1200
y_offset_bus = 600
x_offset_car2 = 600
y_offset_car2 = 700
frame[y_offset_car:y_offset_car+car.shape[0], x_offset_car:x_offset_car+car.shape[1]] = car
frame[y_offset_bus:y_offset_bus+bus.shape[0], x_offset_bus:x_offset_bus+bus.shape[1]] = bus
frame[y_offset_car2:y_offset_car2+car2.shape[0], x_offset_car2:x_offset_car2+car2.shape[1]] = car2
cv2.imwrite("Generated_frame.png", frame)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

True

In [120]:
# Initialize video parameters
width = 1920
height = 1080
FPS = 24
seconds = 8

In [ ]:
fourcc = VideoWriter_fourcc(*'MP42')
video = VideoWriter('./HUD.mp4', fourcc, float(FPS), (width, height))
for _ in range(FPS * seconds):
    frame = cv2.imread(filename='white_background.jpg')
    frame = cv2.resize(frame,(1920,1080))
    car = cv2.imread(filename='car.png')
    car = cv2.resize(car,(240,300))
    bus = cv2.imread(filename='bus.png')
    bus = cv2.resize(bus,(220,380))
    car2 = cv2.imread(filename='car2.png')
    car2 = cv2.resize(car2,(180,300))
    y_offset_car -= 3
    y_offset_bus -= 1
    y_offset_car2 -= 2
    frame[y_offset_car:y_offset_car+car.shape[0], x_offset_car:x_offset_car+car.shape[1]] = car
    frame[y_offset_bus:y_offset_bus+bus.shape[0], x_offset_bus:x_offset_bus+bus.shape[1]] = bus
    frame[y_offset_car2:y_offset_car2+car2.shape[0], x_offset_car2:x_offset_car2+car2.shape[1]] = car2
    video.write(frame)
video.release()